In [18]:
import pandas as pd
import numpy as np
file = "Resources/purchase_data.csv"
df = pd.read_csv(file)
df.head()
new_df = df
print(new_df.head())


   Purchase ID             SN  Age Gender  Item ID  \
0            0        Lisim78   20   Male      108   
1            1    Lisovynya38   40   Male      143   
2            2     Ithergue48   24   Male       92   
3            3  Chamassasya86   24   Male      100   
4            4      Iskosia90   23   Male      131   

                                   Item Name  Price  
0  Extraction, Quickblade Of Trembling Hands   3.53  
1                          Frenzied Scimitar   1.56  
2                               Final Critic   4.88  
3                                Blindscythe   3.27  
4                                       Fury   1.44  


In [19]:
#Total players

TotalPlayers = len(df["SN"].value_counts())
TotalPlayers_df = pd.DataFrame(columns = ['Total Players'])
TotalPlayers_df.loc[0,'Total Players'] = TotalPlayers
TotalPlayers_df

,Total Players
0,576


In [20]:
#Summary of Unique Items, Average Price, Number of Purchases, and Total Revenue
#Creating the necessary columns, then making a new df 
item_count = len(df["Item Name"].value_counts()) 
avg_price = df['Price'].mean()
purchases = df['Price'].count()
revenue = df['Price'].sum()
summary_df = pd.DataFrame({"Number of Unique Items": item_count,
                              "Average Price": avg_price,
                              "Number of Purchases": purchases,
                              "Total Revenue": revenue},index=[0])
print(summary_df)

   Number of Unique Items  Average Price  Number of Purchases  Total Revenue
0                     179       3.050987                  780        2379.77


In [21]:
# Drop all rows with missing information
summary_df = summary_df.dropna(how='any')
summary_df

Gender_df = df.set_index("Gender")
Gender_df.head()

gender2 = df.loc[:, ["Gender", "SN", "Age"]]
gender3 = gender2.drop_duplicates()
gender_group = gender3.groupby(["Gender"])
gender_group_df = gender_group.count() # new df
renamed_df = gender_group_df.rename(columns={"SN":"Total Counts"}) # new df


In [22]:
gender_percentage = renamed_df["Total Counts"] / TotalPlayers
formatted_percentage = round(gender_percentage,2) * 100
#format 'Age' as a percentage
renamed_df['Age'] = formatted_percentage
renamed_df
# change column names to correct labels
renamed_df.rename(columns={'Age':'Percentage of Players'})

,Total Counts,Percentage of Players
Gender,,
Female,81,14.0
Male,484,84.0
Other / Non-Disclosed,11,2.0


In [24]:
gender4 = df.loc[:, ["Gender","SN","Price"]]
gender_group2 = gender4.groupby(["Gender"])
gender_group2_df = gender_group2.count()
renamed2_df = gender_group2_df.rename(columns={"SN":"Purchase Count"})
avg_purchase_price = round(gender_group2['Price'].mean(),2)
total_purchase = round(gender_group2['Price'].sum(),2)
avg_total_purchase_per = round(total_purchase / renamed_df['Total Counts'],2)
avg_total_purchase_per
summary_df2 = pd.DataFrame({ "Purchase Count": renamed2_df['Purchase Count'],
                             "Average Purchase Price": avg_purchase_price,
                              "Total Purchase Value": total_purchase,
                             "Avg Total Purchase per Person":avg_total_purchase_per})
summary_df2

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [13]:
age_bins = ([0,9,14,19,24,29,34,39,999])
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
gender3["Total Count"] = pd.cut(gender3["Age"], age_bins, labels=labels, include_lowest=True)
Total_Count_df = pd.DataFrame(gender3["Total Count"].value_counts())
Total_Count_df["Percentage of Players"] = ((Total_Count_df["Total Count"]/(Total_Count_df["Total Count"].sum())) * 100).round(2)


sorted_df = Total_Count_df.sort_index()

sorted_df

C:\Users\winyi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [8]:
#purchasing age analysis - columns = Purchase Coutn, Avg Purchase Price, Total Purchase Price, Avg Total Purchase per Person
df['Age Ranges'] = pd.cut(df['Age'], age_bins, labels=labels, include_lowest=True)
age_analysis = df.groupby('Age Ranges')                        
age_analysis.head()

pur_count = age_analysis['Purchase ID'].count()
avg_pur_price = (age_analysis['Price'].mean()).round(2)
tot_pur_val = age_analysis['Price'].sum()

# create new df
analysis_summary = pd.DataFrame({"Purchase Count": pur_count,
                                 "Average Purchase Price": avg_pur_price,
                                 "Total Purchase Value": tot_pur_val})
print(analysis_summary)


            Purchase Count  Average Purchase Price  Total Purchase Value
Age Ranges                                                              
<10                     23                    3.35                 77.13
10-14                   28                    2.96                 82.78
15-19                  136                    3.04                412.89
20-24                  365                    3.05               1114.06
25-29                  101                    2.90                293.00
30-34                   73                    2.93                214.00
35-39                   41                    3.60                147.67
40+                     13                    2.94                 38.24


In [9]:
#Top 5 spenders
# create columns
spender =df.groupby('SN')

top_count=spender['Purchase ID'].count()

avg_price=spender['Price'].mean()

total_purchase =spender['Price'].sum()
#add them to new df
top5_df =pd.DataFrame({'Purchase Count': top_count,
                        'Average Purchase' : avg_price,
                        'Total Purchase Value' : total_purchase})
top5_df.style.format({'Average Purchase Price':"${:,.2f}", 'Total Purchase Value':"${:,.2f}"})
#use nlargest to return top 5
top_5 = top5_df.nlargest(5,'Total Purchase Value')
top_5.style.format({'Average Purchase Price':"${:,.2f}", 'Total Purchase Value':"${:,.2f}"})
print(top_5)

             Purchase Count  Average Purchase  Total Purchase Value
SN                                                                 
Lisosia93                 5          3.792000                 18.96
Idastidru52               4          3.862500                 15.45
Chamjask73                3          4.610000                 13.83
Iral74                    4          3.405000                 13.62
Iskadarya95               3          4.366667                 13.10


In [10]:
#Most Popular Items
 
#Retrieve the Item ID, Item Name, and Item Price columns
#Group by Item ID and Item Name. obtain purchase count, item price, and total purchase value

popular = df[['Item ID', 'Item Name', 'Price']]
group = popular.groupby(['Item ID', 'Item Name'])
count = group['Item ID'].count()
item_price = (group['Price'].sum()/count).round(2)
price = group['Price'].sum()

top5_df = pd.DataFrame({'Purchase Count': count,
                       'Price': item_price,
                       'Total Purchase Value': price})
top5 = top5_df.nlargest(5,'Purchase Count')
print(top5)
                                      

                                                      Purchase Count  Price  \
Item ID Item Name                                                             
92      Final Critic                                              13   4.61   
178     Oathbreaker, Last Hope of the Breaking Storm              12   4.23   
82      Nirvana                                                    9   4.90   
108     Extraction, Quickblade Of Trembling Hands                  9   3.53   
132     Persuasion                                                 9   3.22   

                                                      Total Purchase Value  
Item ID Item Name                                                           
92      Final Critic                                                 59.99  
178     Oathbreaker, Last Hope of the Breaking Storm                 50.76  
82      Nirvana                                                      44.10  
108     Extraction, Quickblade Of Trembling Hands            

In [11]:
#Most Profitable

profit5 = top5_df.nlargest(5,'Total Purchase Value')
print(profit5)

                                                      Purchase Count  Price  \
Item ID Item Name                                                             
92      Final Critic                                              13   4.61   
178     Oathbreaker, Last Hope of the Breaking Storm              12   4.23   
82      Nirvana                                                    9   4.90   
145     Fiery Glass Crusader                                       9   4.58   
103     Singed Scalpel                                             8   4.35   

                                                      Total Purchase Value  
Item ID Item Name                                                           
92      Final Critic                                                 59.99  
178     Oathbreaker, Last Hope of the Breaking Storm                 50.76  
82      Nirvana                                                      44.10  
145     Fiery Glass Crusader                                 